### Heap Data structure

In this notebook we will look at heap data structure and come up with a simple implementation in Python

We have seen Queues and Stack before which are support FIFO(First In First Out) and LIFO(Last In First Out) ordering of elements added to these data structures respectively. They are both used in BFS and DFS traversal of graph/trees respectively. We will now look at a another special type of data structure called Heap which has a very typical usecases.

When choosing a Data structure its important to think which operation we will perfrom frequently. For example, in Djikstra's algorithm, a part of it goes through all edges and vertices (m + n) gives us a complexity of $\theta(m + n)$, however, find the next vertex with lowest Djikstra score requires $\theta(n)$ and thus the entire algorithm has complexity $\theta{((m + n) \cdot n)}$ which is quadratic. Imagine we have an algorithm which gives us this next vertex to pick in $\theta(log(n))$, then our algorithm's complexity is $\theta((m + n) \cdot log(n))$ which is way faster than quadratic complexity. As choosing the next vertex in Djikstra's algorothm is a frequent operation, making it run faster makes the entire algorithm run faster.

With this in mind, lets define the heap data structure


---

Heap data structure lets us maintain the minimum/maximum value of an evolving set of objects.

The key here is the word **evolving**. Finding the minimum/maximum from a fixed set of values can be done in linear time. However maintaining the minimum and maximum from an evolving stream of objects supporting two operations **Extract-Min**(or Extract-Max) and **Insert** is not straight forward.
We may think of sorting the numbers, and lets look at the time complexity of these operations

- Case 1, Sorting the array
    - Extract-Min: The time complexity if extracting the min value from a sorted set is $\theta(1)$
    - Insert: Initial operation with a list of numbers will require $\theta(n \cdot log(n))$ with each subsequent insert requiring linear time $\theta(n)$
    
- Case 2: Keeping unordered linked list

    - Extract-Min: Scanning the unordered linked list to extract the minimum will take $\theta(n)$
    - Insert: This straightforward and we just add the object to the end of the linked list in $\theta(1)$
    
As we can see, both options has a linear time operation for either insert of extract and what we need is s datastructure that allows both these operations to be performed much faster than linear time.

The Heap Datastructure will give us the following running time guarantees

|Operation|Complexity|
|:-|-:|
|Insert|$\theta(logn)$ |
|Extract-Min|$\theta(logn)$ |
|Find-Min|$\theta(1)$ |
|Delete|$\theta(logn)$ |
|Heapify|$\theta(n)$ |


Naive implementation of Find-Min simply extracts min and inserts it back in $\log(n)$ time but we will implement the datastructure which will do it in constant time

Similarly, heapify can simply sort the input in $\theta(nlog(n))$ time (or perform Insert on all n elements) but we will see how we can heapify the unordered array in linear time.

Before we implement this datastructure, lets look at a very good use of it. Let's start with Selection sort algorithm which we will implement below.

In [13]:
def selectionSort(arr):
    l = len(arr)
    for i in range(l - 1):        
        #Find Minimum from i + 1 to the end
        minIdx = i + 1  
        for j in range(i + 1, l):
            if arr[j] < arr[minIdx]:
                minIdx = j
        #Swap if element at i is greater than the minimum value starting at i + 1
        if arr[i] > arr[minIdx]:
            arr[i], arr[minIdx] = arr[minIdx], arr[i]
        
a = [2, 4, 1, 6, 9, 7, 3, 5, 8]
print('Before Sorting', a)
selectionSort(a)
print('After Sorting', a)

Before Sorting [2, 4, 1, 6, 9, 7, 3, 5, 8]
After Sorting [1, 2, 3, 4, 5, 6, 7, 8, 9]


As we see above, selection sort scans all elements after the index i to find the minimum value after the index at i and swaps the minimim found at after i with i if we find one. Thus in first iteration we have n comparisons and subsequent comparisons are 1 less then previous iteration, Therefore the number of comparisons for an array of size n is n + (n - 1) + (n - 2) + ... 1 = $\frac{(n)(n + 1)}{2}$ which is $\theta(n^2)$

As we can see the most frequent operation we do is find the minimum starting at an index. We therefore see a good use of heap here where initially heapify the array in linear time and then keep extracting the minimum element in $\theta(log(n))$ n times giving us the time complexity of $\theta(n \cdot log(n))$

We also know that no comparison based sorting algorithm can perform better than $\theta(n \cdot log(n))$, which also means heap cannot perform Extract-Min better than $\theta(log(n))$ as any better complexity will give us the time complexity of the sorting algorithm better than $\theta(n \cdot log(n))$ which is not possible.

---
** Quiz 10.1**

The answer of (b), $\theta(n \cdot log(n))$

---

One application of Heaps is median maintenance. The goal of this problem is to find the median of the given stream of numbers. Finding median of a static list of numbers if not difficult. However, doing so for a stream of numbers efficiently requires us to use two Heaps. Let us write a Python implementation of this problem. Since we havent implemented heaps ourselves, we will use the standard Python package for heaps ``heapq``

In [25]:
class StreamMedian:
    

    
    def __init__(self):
        self.minHeap, self.maxHeap = [], []
        
    def addNumber(self, number):
        import heapq
        
        # New numbers are added to the left half or right half (Max heap or Min Heap) 
        # by comparing the valued in the heap in constant time
        if len(self.maxHeap) == 0 or number > self.minHeap[0]:
            heapq.heappush(self.minHeap, number)
        else:
            heapq.heappush(self.maxHeap, -number)
        
        # ------------  ------------
        # |  Max Heap|  |  Min Heap|  
        # ------------  ------------
        # Two heaps are used as above, the values prior to median are in the Max heap on the left  
        # and those after the median, including the median in case odd numbers are in the min heap on the right 
        # We maintain the length of min heap no more than 1 greater than max heap. In case of total even numbers
        # the lengths of both heaps are same, in case of total odd numbers, the min heap will have one element more
        # than the max heap. The below two loops het us maintain this invariant.
        # 
        
        
        while len(self.maxHeap) > len(self.minHeap):
            heapq.heappush(self.minHeap, -heapq.heappop(self.maxHeap))
            
        while len(self.minHeap) - len(self.maxHeap) > 1:
            heapq.heappush(self.maxHeap, -heapq.heappop(self.minHeap))
        
            
    def median(self):        
        if len(self.minHeap) != len(self.maxHeap):
            #Odd number of elements, the top of the Min priority queue (right half) is the median
            return self.minHeap[0]
        else:
            #Even elements, the median is mean of the top of left and right half heaps
            return (self.minHeap[0] - self.maxHeap[0]) / 2
        
s = StreamMedian()

nums = [9, 10, 6, 2, 7, 1, 5, 8, 3, 4]
for i in range(len(nums)):
    s.addNumber(nums[i])
    print('Added numbers', sorted(nums[: i + 1]), ', median is,', s.median())

Added numbers [9] , median is, 9
Added numbers [9, 10] , median is, 9.5
Added numbers [6, 9, 10] , median is, 9
Added numbers [2, 6, 9, 10] , median is, 7.5
Added numbers [2, 6, 7, 9, 10] , median is, 7
Added numbers [1, 2, 6, 7, 9, 10] , median is, 6.5
Added numbers [1, 2, 5, 6, 7, 9, 10] , median is, 6
Added numbers [1, 2, 5, 6, 7, 8, 9, 10] , median is, 6.5
Added numbers [1, 2, 3, 5, 6, 7, 8, 9, 10] , median is, 6
Added numbers [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] , median is, 5.5



We will now look at an application of heaps to implement Djikstra Algorithm. Recall that in Djikstra's algorithm the crux was to pick the next vertex with lowest Djikstra score greedily. In absence of datastructure like Heap, we have to look at all edges where one vertex has its Djikstra's score calculated and another one lies outside the frontier. This is an expensive operation and the algorithm in absence of heap runs in polynomial time in worst case.
Lets implement heap using `heapq` package in the following code snippet. First, lets define some functions to read the underlying files storing graph edges and vertices

In [28]:
def parseFileToAdjList(fname):
    # Constructs an undirected graph from the file representation of the 
    with open(fname, 'r') as f:
        lines = f.readlines()
        
    def rawToAdjList(l):
        splits = l.strip().split('\t')
        source, *dest = splits
        return (source, list(map(lambda d: d.split(','), dest)))
    
    lines = dict(map(rawToAdjList, lines))
    
    return lines
    
adj_list = parseFileToAdjList('problem9.8test.txt')
adj_list

{'1': [['2', '1'], ['8', '2']],
 '2': [['1', '1'], ['3', '1']],
 '3': [['2', '1'], ['4', '1']],
 '4': [['3', '1'], ['5', '1']],
 '5': [['4', '1'], ['6', '1']],
 '6': [['5', '1'], ['7', '1']],
 '7': [['6', '1'], ['8', '1']],
 '8': [['7', '1'], ['1', '2']]}

In [50]:
def djikstra_heapq(adj_list, source):
    from heapq import heappush as push, heappop as pop
    h, visited, res = [], set(), {v : float('inf') for v in adj_list}
    push(h, (0, source))
    while len(h) > 0:
        djikstraScore, vertex = pop(h)
        if vertex in visited:
            continue
        visited.add(vertex)
        res[vertex] = djikstraScore
        for a, d in adj_list[vertex]:
            push(h, (djikstraScore + int(d), a))
    
    return res

In [51]:
source = '1'
res = djikstra_heapq(adj_list, source)
for s in res:
    print('Vertex', s, 'is at a distance', res[s], 'from source', source)

Vertex 1 is at a distance 0 from source 1
Vertex 2 is at a distance 1 from source 1
Vertex 3 is at a distance 2 from source 1
Vertex 4 is at a distance 3 from source 1
Vertex 5 is at a distance 4 from source 1
Vertex 6 is at a distance 4 from source 1
Vertex 7 is at a distance 3 from source 1
Vertex 8 is at a distance 2 from source 1



The results look good, now lets run the bigger file and see how long it takes to run the algorithm, first we load the file

In [52]:
%%time
adj_list = parseFileToAdjList('problem9.8.txt')

CPU times: user 4.93 ms, sys: 1.87 ms, total: 6.8 ms
Wall time: 31.3 ms


In [56]:
%%time
source = '1'
res = djikstra_heapq(adj_list, source)

CPU times: user 8.51 ms, sys: 505 µs, total: 9.01 ms
Wall time: 8.57 ms


In [57]:
vertex_subset = [7,37,59,82,99,115,133,165,188,197]
for s in vertex_subset:
    print('Vertex', s, 'is at a distance', res[str(s)], 'from source', source)

Vertex 7 is at a distance 2599 from source 1
Vertex 37 is at a distance 2610 from source 1
Vertex 59 is at a distance 2947 from source 1
Vertex 82 is at a distance 2052 from source 1
Vertex 99 is at a distance 2367 from source 1
Vertex 115 is at a distance 2399 from source 1
Vertex 133 is at a distance 2029 from source 1
Vertex 165 is at a distance 2442 from source 1
Vertex 188 is at a distance 2505 from source 1
Vertex 197 is at a distance 3068 from source 1


The above results looks good. Recall when we implemented `djikstra_naive` in Chapter 9, following was the running time for this algorithm giving identical results

```
    CPU times: user 189 ms, sys: 4.49 ms, total: 193 ms
    Wall time: 194 ms
```

As we can see the implementation using heap is over 20 times faster, though for this reasonably small graph the running time is not too much for the naive implementation as well. For large graphs, this difference in performance will be noticable.

We stored tuples in heap as `(DjikstaScore, vertex)` and min heap guaranteed that we always get a vertex with smallest Djikstra Score. We maintain a set of vertices we visited to skip entries we read from Heap in case the vertex has already got a shortest path from source vertex to keep the source code short and consise.

---

**Quiz 10.2**

(b), $\theta(m)$, notice that for the entire graph the lines (given in the algorithm in the text book) will go through all the edges once. Thus the operation on line 13 and 14 execute m number of times.

---

#### Implementation of heaps

In this section we will implement heaps in Python ourself and not use the heapq library we used. Instead, we will test the implementation of our library against heapq to validate its correctness

In [102]:
class Heap:
    
    def __init__(self):
        #To make calculations easy, we add 0 to the heap initially to make the indexing 1 based, that way for any 
        #index, the parent element is at index n // 2 and children for any node n are 2n and 2n + 1, if they exist
        self.heap = [0]
    
    def __bubbleup__(self):
        pos = len(self.heap) - 1
        element = self.heap[pos]
        #Move the element up the heap till the parent of the element is less than the element or we reach the root     
        while pos > 1:
            #Right shift by 1 to faster divide by 2, gives same result as pos // 2         
            parent  = pos >> 1
            if self.heap[parent] > element:
                self.heap[parent], self.heap[pos] = self.heap[pos], self.heap[parent]
                pos = parent
            else:
                break
    
    def min_(self):
        return self.heap[1] if len(self.heap) > 1 else None
        
        
        
    def push(self, element):
        self.heap.append(element)
        self.__bubbleup__()
        
    def __sink__(self, idx = 1):        
        n = len(self.heap)
        while True:
            #Terminate after we reach a state where the children if a node are both less or we reach 
            #a state where the element is a leaf where we cant sink the value any further
            elem = self.heap[idx]
            #Multiple by two is achieved by left shifting the bits by 1
            twoidx = idx << 1
            children = [c for c in  [twoidx, twoidx + 1] if c < n and self.heap[c] < elem]
            #If no children found with values less that the value, we can say the node is in rightful place
            if not children:
                break
            if len(children) == 1:
                #Special case where we have only one child and its value is less than the then node, in which case
                #We simpli swap their positions
                childIdx = children[0]
            else:
                #Case where both children are less than the node's value, in which case we swap the position
                #with the minimum of the two and continue to sink the node till it reaches the rightful position
                c1, c2 = children[0], children[1]
                childIdx = c1 if self.heap[c1] < self.heap[c2] else c2
            
            self.heap[idx], self.heap[childIdx], idx = self.heap[childIdx], self.heap[idx], childIdx
 
    def pop(self):
        top = self.min_()
        if top is not None:
            end = self.heap.pop()
            if len(self.heap) > 1:
                self.heap[1] = end
                self.__sink__()
        return top
        
    def heapify(self, array):
        self.heap += array
        #Start with half of the array as everything after this is a child node of all nodes in the first half 
        #and sink operation on those nodes will have the heap in correct order
        for i in reversed(range(1, len(self.heap) // 2 + 1)):
            self.__sink__(idx = i)
            
    
        

In [105]:
#1 Create empty heap and push elements one by 1, in all, push runs in nlog(n) time
h1 = Heap()
nums = [9, 3, 6, 2, 7, 1, 8, 5, 10, 4]
for n in nums:
    h1.push(n)
    
print('1. Elements taken from heap 1', [h1.pop() for _ in range(len(nums))])

#2. Create an empty heap and heapify the same array we used earlier, this time heapify runs in O(n) time
h2 = Heap()
h2.heapify(nums)
print('2. Elements taken from heap 2', [h2.pop() for _ in range(len(nums))])


1. Elements taken from heap 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
2. Elements taken from heap 2 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]



As we see above, the heapify does exactly what we expect it to do but how do we support the claim that the heapify runs in linear time? Below proof will clarify this.

- A heap of size n will have maximum log(n) levels and the heap is always balanced.
- We have a maximum of n / 2 nodes which form the leaf nodes in a heap. Assuming that the leaves are at height 0 and root at height log(n), we can say that at any given height we have a maximum of $\frac{n}{2^{n + 1}}$ nodes. To illustrate that this formula is correct, consider a tree with three levels (7 nodes). Thus at base (height = 0), we have ${\frac{7}{2^{0 + 1}}}$ which is approximately 4 similarly at height 2 and 1 we will have 2 and 1 nodes respectively.
- The total work done in heapify is therefore work done per node, $\theta(h)$, to heapify times the number of nodes which can be expressed in terms of levels as follows

$\sum_{0}^{log(n)}\frac{n}{2^{n + 1}} \cdot \theta(h) $

Thus the worst case time complexity to heapify is  $\theta(n \cdot \sum_{0}^{log(n)}\frac{h}{2^{n + 1}})$

The term 

$\sum_{0}^{log(n)}\frac{h}{2^{n + 1}} = \frac{1}{2} \cdot \sum_{0}^{log(n)}\frac{h}{2^n}$

The term $\sum_{0}^{\infty}\frac{h}{2^n} = 2$ 

and hence $\sum_{0}^{log(n)}\frac{h}{2^{n + 1}}$ reduces to a constant value and hence $\theta(n \cdot \sum_{0}^{log(n)}\frac{h}{2^{n + 1}}) = \theta(n)$ 



---

##### Test Your Understanding

**Problem 10.1** (b) and (c). This is a classic usecase for heap

**Problem 10.2** (b), in a sorted array, we start from the end shifting the elements to the right by one till we reach a position where the new element is to be inserted, in the worst case we end up shifting all elements on each iteration and thus this is linear time operation. Finding minimum/maximum is constant time operation

**Problem 10.3** (a), in this case inserting an element is simply appending the number to the end of the unsorted array, however finding minimum/maximum requires us to scan through all elements of the array.

**Problem 10.4** (a) ( and (b) by a small tweak in some special cases ), finding fifth largest is a matter of invoking ExtractMin five times and adding them back to heap. Thus the number of operations is constant. (b) is also correct in some cases say numbers as keys where we simply negate the numbers before insert in which case extracting the min once is enough, in fact this can be done without extraction and just reading the top element in heap in constant time. However, in general, finding the max using ExtractMin requires linear operations. Medians require us to extract half the heap out and thus is a linear time operation.

*TODO: Revisit 10.5, 10.6 and 10.7 aftre understanding what finding the  bottleneck means, will also require me to attempt problem 9.7 in previous chapter*

**Problem 10.8** Already attempted in this notebook earlier.